## Campos involucrados

- titulo
- descripcion

## Ideas

- wordcloud
- normalizacion
- stemming

## Hipótesis

- ciertas palabras indican mayor precio (luminoso, jardín, hermoso, vista...)
- a más palabras, mayor precio

In [3]:
#importo las funciones para levantar los dataframes
%run "../../utils/dataset_parsing.ipynb"
#importo las funciones para graficar
%run "../../utils/graphs.ipynb"
df = levantar_datos("../../"+DATASET_RELATIVE_PATH)
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'mes', 'ano', 'dia'],
      dtype='object')

In [4]:
import nltk  
from nltk.corpus import stopwords  
from string import punctuation  

In [5]:
spanish_stopwords = set(stopwords.words('spanish'))
non_words = set(punctuation)
non_words.update({'¿', '¡'})
non_words.update(map(str,range(10)))

In [79]:
def add_to_dict(word, dictionary):
    dictionary[word] = dictionary.get(word, 0) + 1 

def get_word_count(series):
    """
    
    """
    words_count = {}
    no_words_count = {}
    for title in series.values:
        if not isinstance(title,str): continue
        words = title.split()
        for word in words:
            text = ''.join([" " if c in non_words else c for c in word]).strip()
            if text in spanish_stopwords: 
                add_to_dict(text, no_words_count)
                continue
            add_to_dict(text, words_count)
    return words_count, no_words_count

In [116]:
from collections import Counter

def get_word_counters(series):
    """
        Faltaría analizar stemming
    """
    words_counter = Counter()
    no_words_counter = Counter()
    for title in series.values:
        if not isinstance(title,str): continue
        words = filter(lambda x: len(x)>2, title.split())
        for word in words:
            text = ''.join([" " if c in non_words else c for c in word]).strip()
            text_ok = len(text) > 2 and not text in spanish_stopwords
            counter_to_update = words_counter if text_ok else no_words_counter 
            counter_to_update.update({text})
    return words_counter, no_words_counter

In [117]:
titulo_palabras, titulo_no_palabras = get_word_counters(df["titulo"])

In [119]:
descripcion_palabras, descripcion_no_palabras = get_word_counters(df["descripcion"])

In [120]:
print(len(titulo_palabras), len(titulo_no_palabras), len(descripcion_palabras), len(descripcion_no_palabras))

25477 406 174523 585


In [131]:
titulo_palabras.most_common(100)

[('casa', 121528),
 ('venta', 104820),
 ('departamento', 38162),
 ('san', 16410),
 ('excelente', 12397),
 ('lomas', 11088),
 ('terreno', 10608),
 ('col', 10349),
 ('residencial', 10035),
 ('hermosa', 9299),
 ('condominio', 8538),
 ('valle', 7861),
 ('oportunidad', 6847),
 ('santa', 6844),
 ('fracc', 6272),
 ('fraccionamiento', 6028),
 ('nueva', 5896),
 ('casas', 5847),
 ('remate', 5602),
 ('bonita', 5445),
 ('recamaras', 5426),
 ('cerca', 5096),
 ('zona', 4983),
 ('colonia', 4877),
 ('residencia', 4776),
 ('real', 4608),
 ('privada', 4188),
 ('centro', 3897),
 ('jardines', 3768),
 ('depto', 3711),
 ('vendo', 3631),
 ('vista', 3512),
 ('sur', 3448),
 ('nuevo', 3289),
 ('departamentos', 3268),
 ('amplia', 3220),
 ('preciosa', 3195),
 ('juriquilla', 3107),
 ('ciudad', 3097),
 ('vende', 3074),
 ('norte', 3061),
 ('comercial', 2989),
 ('villa', 2956),
 ('méxico', 2920),
 ('hacienda', 2848),
 ('cumbres', 2826),
 ('planta', 2822),
 ('juárez', 2773),
 ('local', 2634),
 ('bosques', 2619),
 ('bo

In [130]:
descripcion_palabras.most_common(100)

[('baño', 185496),
 ('cocina', 155094),
 ('casa', 148033),
 ('sala', 129318),
 ('recamaras', 114087),
 ('comedor', 108051),
 ('cuenta', 88651),
 ('p  p', 82425),
 ('servicio', 80654),
 ('dos', 76341),
 ('cuarto', 72810),
 ('planta', 70813),
 ('baños', 70715),
 ('completo', 66226),
 ('principal', 61872),
 ('estacionamiento', 61793),
 ('excelente', 58634),
 ('integral', 58423),
 ('lavado', 58311),
 ('autos', 55402),
 ('patio', 53690),
 ('área', 50664),
 ('vestidor', 49661),
 ('terreno', 46634),
 ('departamento', 45781),
 ('recámaras', 45639),
 ('terraza', 43879),
 ('closet', 43754),
 ('cochera', 43275),
 ('jardín', 40586),
 ('piso', 39404),
 ('baja', 37862),
 ('vigilancia', 37638),
 ('venta', 36143),
 ('cerca', 35357),
 ('recamara', 35016),
 ('alta', 33990),
 ('amplia', 33479),
 ('acabados', 32928),
 ('zona', 32708),
 ('medio', 32505),
 ('estudio', 30702),
 ('equipada', 30670),
 ('niveles', 29301),
 ('acceso', 27453),
 ('alberca', 26676),
 ('vista', 26648),
 ('area', 26144),
 ('tres', 25

In [125]:
import matplotlib.pyplot as plt

In [128]:
titulo_palabras_top_200 = titulo_palabras.most_common(200)